In [3]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame,read_csv
from sklearn.utils import shuffle

In [4]:
df = read_csv('heart_failure.csv')
df.head(5)


FileNotFoundError: [Errno 2] No such file or directory: 'heart_failure.csv'

In [ ]:
df['anaemia'] = np.where(df['anaemia'] == 1 ,True,False)
df['diabetes'] = np.where(df['diabetes'] == 1, True, False)
df['high_blood_pressure'] = np.where(df['high_blood_pressure'] == 1, True, False)
df['smoking'] = np.where(df['smoking'] == 1, True, False)
df['sex'] = np.where(df['sex'] == 1, 'Male','Female')


In [ ]:
df.info()

In [ ]:
X = df.drop('DEATH_EVENT', axis = 1)
y = df['DEATH_EVENT']
print(X.shape)
X.head(2)

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTENC
smote = SMOTENC(random_state=1,categorical_features=[0,1,3,5,9,10])
x_bal, y_bal = smote.fit_sample(X,y)
x_bal = DataFrame(x_bal, columns = X.columns)
print(Series(y_bal).value_counts())

In [ ]:
x = pd.get_dummies(x_bal,columns = ['sex'], drop_first = True)
print(x.shape)
x.head()

In [ ]:
col = ['age','creatinine_phosphokinase','ejection_fraction',
       'platelets','serum_creatinine','serum_sodium','time',
       'anaemia','diabetes','high_blood_pressure','smoking',
       'sex_Male']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

col_trans = ColumnTransformer(remainder='passthrough',
                              transformers = [('scaler',MinMaxScaler(),[0,2,4,6,7,8,10])])
trans = col_trans.fit_transform(x)
trans = DataFrame(trans,columns = col)
trans.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(trans,y_bal, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
grid = GridSearchCV(RandomForestClassifier(),
                {'max_depth':range(1,15), 'n_estimators': [50, 100,300],
                'max_features': ['auto', 'sqrt', 'log'], 
                'min_samples_split': [2,3,5,7,9],'min_samples_leaf': [1,2,4,6,8]},
                 cv = 3, n_jobs = 2, scoring = 'recall', verbose = 1)

grid.fit(x_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_features= 'auto',random_state = 3,
                                    min_samples_leaf = 1,
                                    min_samples_split = 2,n_estimators = 100)
classifier.fit(x_train, y_train)

In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, recall_score, accuracy_score,precision_score, f1_score, confusion_matrix
recall = recall_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
f1score = f1_score(y_test,y_pred)
print('the Recall for tuned RandomForest is:{}'.format(round(recall,4)))
print(f'the Accuracy for tuned RandomForest is:{round(accuracy,4)}')
print('the Precision for tuned RandomForest is: %s' %(round(precision,4)))
print(f'the F1_score for tuned RandomForest is:{round(f1score,4)}')

In [ ]:
report = classification_report(y_test,y_pred)
print(report)

In [ ]:
matrix = confusion_matrix(y_test,y_pred)
print(matrix)

In [ ]:
import pickle
pickle.dump(classifier, open('model.pkl','wb'))